```import os
from dotenv import load_dotenv

load_dotenv()

MY_ENV_VAR = os.getenv('MY_ENV_VAR')

print(MY_ENV_VAR)
```

In [2]:
%load_ext dotenv
%dotenv

In [ ]:
import os

assert 'OPENAI_API_KEY' in os.environ.keys(), "Missing 'OPENAI_API_KEY' from env"
assert 'REDDIT_API_CLIENT_SECRET' in os.environ.keys(), "Missing 'REDDIT_API_CLIENT_SECRET' from env"
assert 'REDDIT_API_CLIENT_ID' in os.environ.keys(), "Missing 'REDDIT_API_CLIENT_ID' from env"

AssertionError: Missing 'REDDIT_API_CLIENT_ID' from env

In [ ]:
import praw

# Configure the Reddit instance
reddit = praw.Reddit(
    client_id=os.environ['REDDIT_API_CLIENT_ID'],
    client_secret=os.environ['REDDIT_API_CLIENT_SECRET'],
    user_agent="YOUR_USER_AGENT"
)

# Choose a subreddit
subreddit_name = "learnpython"  # Replace with the subreddit you're interested in
subreddit = reddit.subreddit(subreddit_name)

# Fetch the top 5 posts
print(f"Fetching top 5 posts from r/{subreddit_name}")
for post in subreddit.hot(limit=5):  # Use .top(), .new(), etc., for different sorting
    print(f"\nPost Title: {post.title}")
    print(f"Post ID: {post.id}")
    print(f"Score: {post.score}")
    print(f"URL: {post.url}")

    # Fetch comments for the post
    print("\nComments:")
    post.comments.replace_more(limit=0)  # Remove "load more comments"
    for comment in post.comments.list()[:5]:  # Limit to the first 5 comments
        print(f"- {comment.body[:100]}...")  # Print first 100 characters of each comment
        # print(comment.)

Fetching top 5 posts from r/learnpython

Post Title: Ask Anything Monday - Weekly Thread
Post ID: 1i5d1s3
Score: 3
URL: https://www.reddit.com/r/learnpython/comments/1i5d1s3/ask_anything_monday_weekly_thread/

Comments:
- Ok I want to learn how to code, I have for a while but life has just been to busy, I finally have th...
- Ok, so I’m completely new to python. I just downloaded it and every time I go to type anything it ju...
- I'm writing a silly little script just for the sake of learning python,  that lets me input 2 values...
- I’ve been working on a project that I’d love to get your feedback on. It is a collection of four Pyt...
- silly question sorry but is a .py file always just raw code, no output? If i wanted to export code a...

Post Title: How would you recommend to learn Python if you’re completely new to coding?
Post ID: 1i9n1cq
Score: 26
URL: https://www.reddit.com/r/learnpython/comments/1i9n1cq/how_would_you_recommend_to_learn_python_if_youre/

Comments:
- People usual

KeyboardInterrupt: 

In [33]:
posts = list(subreddit.hot(limit=2))

post_comments = posts[0].comments
posts[0].author.name

'AutoModerator'

In [22]:
posts[0].created_utc

1737331266.0

In [13]:
from pydantic import BaseModel
from datetime import datetime

class RedditComment(BaseModel):
    id:str
    text: str
    score: int
    author_name: str 
    creation_time_utc: str
    
    
class RedditPost(BaseModel):
    title: str
    text: str
    score: int
    creation_time_utc : str
    id: str
    url: str
    author_name: str
    comments: list[RedditComment]
   

In [ ]:
from datetime import timezone
import praw
import os

# Configure the Reddit instance
reddit = praw.Reddit(
    client_id=os.environ['REDDIT_API_CLIENT_ID'],
    client_secret=os.environ['REDDIT_API_CLIENT_SECRET'],
    user_agent="YOUR_USER_AGENT"
)

# Choose a subreddit
subreddit_name = "msp"  # Replace with the subreddit you're interested in
subreddit = reddit.subreddit(subreddit_name)

# Fetch the top 5 posts
posts = []
print(f"Fetching top 5 posts from r/{subreddit_name}")
for post in subreddit.hot(limit=5):  # Use .top(), .new(), etc., for different sorting
    creation_time_utc = datetime.fromtimestamp(post.created_utc, tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S')

    print(f"\nPost Title: {post.title}")
    print(f"Post ID: {post.id}")
    print(f"Score: {post.score}")
    print(f"URL: {post.url}")

    # Fetch comments for the post
    comments = []
    print("\nComments:")
    post.comments.replace_more(limit=0)
    for comment in post.comments.list():
        print(f"- {comment.body[:100]}...")
        comment_creation_time = datetime.fromtimestamp(comment.created_utc, tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S')
        reddit_comment = RedditComment(
            id=comment.id,
            text=comment.body,
            score=comment.score,
            author_name=comment.author.name,
            creation_time_utc=comment_creation_time
        )
        comments.append(reddit_comment)
        # print(comment.)
    
    
    
    reddit_post = RedditPost(
        title=post.title,
        id=post.id,
        score=post.score,
        url=post.url,
        text=post.selftext,
        creation_time_utc=creation_time_utc,
        author_name=post.author.name,
        comments=comments
    )
    
    posts.append(reddit_post)

Fetching top 5 posts from r/msp

Post Title: Weekly Promo and Webinar Thread
Post ID: 1i5inna
Score: 6
URL: https://www.reddit.com/r/msp/comments/1i5inna/weekly_promo_and_webinar_thread/

Comments:


AttributeError: 'CommentForest' object has no attribute 'hot'

In [15]:
[p.model_dump() for p in posts]

[{'title': 'Weekly Promo and Webinar Thread',
  'text': "Vendors, please put self-promoting posts or webinar information in this thread. Threads that are posted elsewhere will be removed.\n\n**Please do not use URL shorteners. Reddit doesn't like these and your posts will be automatically removed by the auto moderator. Only include direct posts to your site.**\n\nIt's fine to post if you did last week - if the group doesn't want to see it again, your comment will just get downvoted :)",
  'score': 6,
  'creation_time_utc': '2025-01-20 05:01:36',
  'id': '1i5inna',
  'url': 'https://www.reddit.com/r/msp/comments/1i5inna/weekly_promo_and_webinar_thread/',
  'author_name': 'AutoModerator',
  'comments': [{'id': 'm84ggvd',
    'text': '# [MSP Partner Program. Boost Your Sales And Profits.](https://easydmarc.com/solutions/dmarc-managed-service-providers?utm_source=social&utm_medium=reddit&utm_campaign=MSP)\n\nDo you know that?\n\n* MSPs emphasizing DMARC compliance as a key component of the

In [ ]:
p = posts[0]
p.model_dump_json(indent=4)


'{\n    "title": "Weekly Promo and Webinar Thread",\n    "text": "Vendors, please put self-promoting posts or webinar information in this thread. Threads that are posted elsewhere will be removed.\\n\\n**Please do not use URL shorteners. Reddit doesn\'t like these and your posts will be automatically removed by the auto moderator. Only include direct posts to your site.**\\n\\nIt\'s fine to post if you did last week - if the group doesn\'t want to see it again, your comment will just get downvoted :)",\n    "score": 6,\n    "creation_time_utc": "2025-01-20 05:01:36",\n    "id": "1i5inna",\n    "url": "https://www.reddit.com/r/msp/comments/1i5inna/weekly_promo_and_webinar_thread/",\n    "author_name": "AutoModerator",\n    "comments": [\n        {\n            "id": "m84ggvd",\n            "text": "# [MSP Partner Program. Boost Your Sales And Profits.](https://easydmarc.com/solutions/dmarc-managed-service-providers?utm_source=social&utm_medium=reddit&utm_campaign=MSP)\\n\\nDo you know t

In [ ]:
from pathlib import Path

Path(r"C:\repos\sales_competitor_analysis\test.json").write_text(p.model_dump_json(indent=4))
p2 = RedditPost.model_validate_json(Path(r"C:\repos\sales_competitor_analysis\test.json").read_text())
    
p2

RedditPost(title='Weekly Promo and Webinar Thread', text="Vendors, please put self-promoting posts or webinar information in this thread. Threads that are posted elsewhere will be removed.\n\n**Please do not use URL shorteners. Reddit doesn't like these and your posts will be automatically removed by the auto moderator. Only include direct posts to your site.**\n\nIt's fine to post if you did last week - if the group doesn't want to see it again, your comment will just get downvoted :)", score=6, creation_time_utc='2025-01-20 05:01:36', id='1i5inna', url='https://www.reddit.com/r/msp/comments/1i5inna/weekly_promo_and_webinar_thread/', author_name='AutoModerator', comments=[RedditComment(id='m84ggvd', text='# [MSP Partner Program. Boost Your Sales And Profits.](https://easydmarc.com/solutions/dmarc-managed-service-providers?utm_source=social&utm_medium=reddit&utm_campaign=MSP)\n\nDo you know that?\n\n* MSPs emphasizing DMARC compliance as a key component of their services have reported 

In [ ]:
from datetime import timezone
import praw
import os

# Configure the Reddit instance
reddit = praw.Reddit(
    client_id=os.environ['REDDIT_API_CLIENT_ID'],
    client_secret=os.environ['REDDIT_API_CLIENT_SECRET'],
    user_agent="YOUR_USER_AGENT"
)

# Choose a subreddit
subreddit_name = "msp"  # Replace with the subreddit you're interested in
query = "Sentinel One"
subreddit = reddit.subreddit(subreddit_name)

# Fetch the top 5 posts
posts = []
print(f"Fetching top 5 posts from r/{subreddit_name}")
for post in subreddit.search(query, sort="relevance", limit=3):
    creation_time_utc = datetime.fromtimestamp(post.created_utc, tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S')

    print(f"\nPost Title: {post.title}")
    print(f"Post ID: {post.id}")
    print(f"Score: {post.score}")
    print(f"URL: {post.url}")

    # Fetch comments for the post
    comments = []
    print("\nComments:")
    post.comments.replace_more(limit=0)  # Remove "load more comments"
    for comment in post.comments.list()[:5]:  # Limit to the first 5 comments
        print(f"- {comment.body[:100]}...")  # Print first 100 characters of each comment
        comment_creation_time = datetime.fromtimestamp(comment.created_utc, tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S')
        reddit_comment = RedditComment(
            id=comment.id,
            text=comment.body,
            score=comment.score,
            author_name=comment.author.name,
            creation_time_utc=comment_creation_time
        )
        comments.append(reddit_comment)
        
    reddit_post = RedditPost(
        title=post.title,
        id=post.id,
        score=post.score,
        url=post.url,
        text=post.selftext,
        creation_time_utc=creation_time_utc,
        author_name=post.author.name,
        comments=comments
    )
    
    posts.append(reddit_post)

In [21]:
subreddit_name = "msp"  # Replace with the subreddit you're interested in
query = "Sentinel One"
subreddit = reddit.subreddit(subreddit_name)

search_results = subreddit.search(query, sort="relevance", limit=3)

In [24]:
next(search_results).title

'Why so much love for Sentinel One?'